In [1]:
#default_exp haystack_search

In [2]:
#export
import pprint
import numpy as np
import pandas as pd
import requests
import torch
from sklearn import metrics
from nltk import tokenize
from operator import itemgetter

from haystack import Finder
from haystack.database.elasticsearch import ElasticsearchDocumentStore
from haystack.database.memory import InMemoryDocumentStore

from haystack.retriever.dense import EmbeddingRetriever
from haystack.utils import print_answers
from pytorch_hackathon import rss_feeds

import seaborn as sns

In [3]:
pd.set_option('max_colwidth', 100)

In [4]:
cm = sns.light_palette("green", as_cmap=True)

In [5]:
%cd ..

/home/kuba/Projects/pytorch_hackathon


In [6]:
!ls data

feeds.txt  topics.txt  zsl_feed_results.csv


In [7]:
rss_feed_urls = list(pd.read_table('data/feeds.txt', header=None).iloc[:,0].values)

In [8]:
feed_df = rss_feeds.get_feed_df(rss_feed_urls)

100%|██████████| 16/16 [00:09<00:00,  1.69it/s]
/home/kuba/Projects/pytorch_hackathon/pytorch_hackathon/rss_feeds.py:63: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 63 of the file /home/kuba/Projects/pytorch_hackathon/pytorch_hackathon/rss_feeds.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  feed_df['text'] = feed_df['summary'].apply(lambda s: bs4.BeautifulSoup(s).text)


In [9]:
#export

use_gpu = torch.cuda.is_available()

In [10]:
pretty_print = pprint.PrettyPrinter(indent=2).pprint

In [11]:
feed_df.head()

,title,title_detail,links,link,summary,summary_detail,id,guidislink,tags,text,...,published_parsed,comments,authors,author,author_detail,updated,updated_parsed,content,href,media_thumbnail
0,Guided Collaborative Training for Pixel-wise Semi-Supervised Learning,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/guided-coll...",https://paperswithcode.com/paper/guided-collaborative-training-for-pixel-wise,"Although SSL methods have achieved impressive results in image classification, the performances ...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/guided-collaborative-training-for-pixel-wise,False,"[{'term': 'Image classification', 'scheme': None, 'label': None}, {'term': 'Image denoising', 's...","Although SSL methods have achieved impressive results in image classification, the performances ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PiNet: Attention Pooling for Graph Classification,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/pinet-atten...",https://paperswithcode.com/paper/pinet-attention-pooling-for-graph,"We propose PiNet, a generalised differentiable attention-based pooling mechanism for utilising g...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/pinet-attention-pooling-for-graph,False,"[{'term': 'Graph classification', 'scheme': None, 'label': None}]","We propose PiNet, a generalised differentiable attention-based pooling mechanism for utilising g...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Rethinking Pseudo-LiDAR Representation,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/rethinking-...",https://paperswithcode.com/paper/rethinking-pseudo-lidar-representation,"Based on this observation, we design an image based CNN detector named Patch-Net, which is more ...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/rethinking-pseudo-lidar-representation,False,NaN,"Based on this observation, we design an image based CNN detector named Patch-Net, which is more ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DensE: An Enhanced Non-Abelian Group Representation for Knowledge Graph Embedding,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/dense-an-en...",https://paperswithcode.com/paper/dense-an-enhanced-non-abelian-group,Capturing the composition patterns of relations is a vital task in knowledge graph completion. <...,"{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/dense-an-enhanced-non-abelian-group,False,"[{'term': 'Entity embeddings', 'scheme': None, 'label': None}, {'term': 'Knowledge graph complet...",Capturing the composition patterns of relations is a vital task in knowledge graph completion. C...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A parallel evaluation data set of software documentation with document structure annotation,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/a-parallel-...",https://paperswithcode.com/paper/a-parallel-evaluation-data-set-of-software,"This paper accompanies the software documentation data set for machine translation, a parallel e...","{

In [12]:
tokenize.wordpunct_tokenize

<bound method RegexpTokenizer.tokenize of WordPunctTokenizer(pattern='\\w+|[^\\w\\s]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>

In [13]:
#export


def _setup_retriever(document_store, model_name, use_gpu, quantize_model):
    retriever = EmbeddingRetriever(
        document_store=document_store,
        embedding_model=model_name,
        use_gpu=use_gpu)
    if not use_gpu and quantize_model:
        quantized_model = torch.quantization.quantize_dynamic(
            retriever.embedding_model.model,
            {torch.nn.Linear}, dtype=torch.qint8
        )
    return retriever



def setup_document_store_with_retriever(
        model_name,
        df,
        text_col='text',
        max_document_length=256,
        use_gpu=use_gpu,
        quantize_model=True):
    embedding_col = text_col + '_emb'
    document_store = InMemoryDocumentStore(
        embedding_field=embedding_col,
    )
    retriever = _setup_retriever(document_store, model_name, use_gpu, quantize_model)
    
    article_embeddings = retriever.embed_queries(
        texts=[
            ' '.join(tokenize.wordpunct_tokenize(text)[:max_document_length])
            for text in df[text_col] 
        ]
    )

    df[embedding_col] = article_embeddings
    document_store.write_documents(df.to_dict(orient='records'))
    return document_store, retriever

In [14]:
article_texts = feed_df['text']

In [15]:
model_name = "deepset/sentence_bert"
document_store, retriever = setup_document_store_with_retriever(model_name, feed_df, 'text', use_gpu)

08/13/2020 21:33:22 - INFO - haystack.retriever.dense -   Init retriever using embeddings of model deepset/sentence_bert
08/13/2020 21:33:22 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
08/13/2020 21:33:22 - INFO - farm.infer -   Could not find `deepset/sentence_bert` locally. Try to download from model hub ...
08/13/2020 21:33:24 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
08/13/2020 21:33:30 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
Inferencing Samples: 100%|██████████| 74/74 [00:10<00:00,  7.01 Batches/s]


In [16]:
feed_df.iloc[1]['text']

'We propose PiNet, a generalised differentiable attention-based pooling mechanism for utilising graph convolution operations for graph level classification. Code: https://github.com/meltzerpete/PiNet'

In [17]:
#export


def doc_to_dict(doc):
    d = {}
    d['text'] = doc.text
    d['title'] = doc.meta['title']
    d['score'] = doc.query_score
    return d

In [18]:
topic_strings = pd.read_table('data/topics.txt', header=None).iloc[:,0].values

In [19]:
print('\n'.join(topic_strings))

deep learning
natural language processing
computer vision
statistics
implementation
visualization
industry
software engineering
reddit question
arxiv
cloud computing
deployment
competitions
business
business intelligence


In [20]:
topic_query_strings = [
    'text is about {}'.format(topic)
    for topic in topic_strings
]

In [21]:
raw_results = retriever.retrieve(
    topic_query_strings[1]
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 32.67 Batches/s]


In [39]:
#export


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def get_scored_df(retriever, raw_results, topic_strings):
    topic_query_strings = [
        'text is about {}'.format(topic)
        for topic in topic_strings
    ]
    
    results = [
        doc_to_dict(doc)
        for doc in raw_results 
    ]
    result_embeddings = np.array([
        doc.meta['text_emb']
        for doc in raw_results
    ]).astype('float32')
    topic_query_embeddings = np.array(retriever.embed_passages(
        list(topic_strings)
    )).astype('float32')
    
    scores_df = pd.DataFrame({})
    scores_df['title'] = list(map(itemgetter('title'), results))
    scores_df['text'] = list(map(itemgetter('text'), results))

    scores = pd.DataFrame(metrics.pairwise.cosine_similarity(
        result_embeddings,
        topic_query_embeddings
    ))
    scores.columns = topic_strings

    scores_df = pd.concat(
        [scores_df, sigmoid(scores)],
        axis=1
    )
    return scores_df

In [40]:
scores_df = get_scored_df(retriever, raw_results, topic_strings)

Inferencing Samples: 100%|██████████| 4/4 [00:00<00:00,  7.14 Batches/s]


In [41]:
scores_df.style.background_gradient(cmap=cm)

In [42]:
[doc.text for doc in raw_results]

['Syntax almost as friendly as Python, while running up to 100x\xa0fasterPhoto Credit(This post was originally published on my personal\xa0blog.)IntroductionIn my last post, [Failure Report] Distill Fine-tuned Transformers into Recurrent Neural Networks, I tried to distill the knowledge of a fine-tuned BERT model into an LSTM or GRU model without any data augmentation and failed to achieve satisfiable results. In the follow-up works, I tried to replicate the easies-to-implement augmentation method\u200a—\u200amasking\u200a—\u200aused in [1] and see its effect. The masking described in [1] is called “whole word masking” [2], that is, masking the whole word instead of just masking a single word\xa0piece.It is non-trivial to implement whole word masking, as it would require the sampling process to be aware of which word piece is itself a whole word, and which is part of a word. As you may know, doing text processing in pure Python is quite slow comparing to other compiled languages. I rec

In [43]:
scores_df

,title,text,deep learning,natural language processing,computer vision,statistics,implementation,visualization,industry,software engineering,reddit question,arxiv,cloud computing,deployment,competitions,business,business intelligence
0,Using Julia to Do Whole Word Masking,"Syntax almost as friendly as Python, while running up to 100x fasterPhoto Credit(This post was o...",0.539542,0.630511,0.533587,0.557276,0.572191,0.569392,0.542997,0.574472,0.551468,0.531546,0.515421,0.542184,0.506937,0.527759,0.543280
1,Transfer Learning with KERAS,"Learning is a never-ending process, but it’s more important to use previously gained knowledge i...",0.696341,0.595428,0.577528,0.566244,0.623856,0.597831,0.579651,0.589617,0.514814,0.502834,0.537744,0.560929,0.568218,0.559038,0.556490
2,How PyTorch Lightning became the first ML framework to runs continuous integration on TPUs,Learn how PyTorch Lightning added CI tests on TPUsContinue reading on PyTorch »,0.676328,0.595533,0.567742,0.567639,0.638649,0.599192,0.585860,0.589025,0.528871,0.514884,0.531069,0.567574,0.562189,0.560442,0.553597
3,Textual Description for Mathematical Equations,Reading of mathematical expression or equation in the document images is very challenging due to...,0.585750,0.561175,0.529714,0.542560,0.564061,0.554569,0.550492,0.519143,0.529417,0.536874,0.509220,0.539236,0.525885,0.546909,0.525588
4,[Notes] “Statistical Inference Enables Bad Science; Statistical Thinking Enables Good Science”,"Reading the Article by Christopher Tong on The American Statistician Volume 73, 2019Photo Credit...",0.585750,0.561175,0.529714,0.542560,0.564061,0.554569,0.550492,0.519143,0.529417,0.536874,0.509220,0.539236,0.525885,0.546909,0.525588
5,A generic pipeline to make offline inferences,"Data Science has been at the core of Doctrine.fr since the beginning. As such, a lot of effort h...",0.583226,0.596984,0.615735,0.660321,0.643634,0.641581,0.584964,0.587327,0.562529,0.558694,0.578877,0.606548,0.583824,0.563006,0.592717
6,"[D] Data Augmentation using Pre-Trained Transformers (BERT, GPT, etc) | Research Paper Walkthrough",Data augmentation is a widely used technique to increase the size of the training data. It helps...,0.583226,0.596984,0.615735,0.660321,0.643634,0.641581,0.584964,0.587327,0.562529,0.558694,0.578877,0.606548,0.583824,0.563006,0.592717
7,Tensorflow Profiler with Custom Training Loop,Analyze and Optimize TensorFlow Performance on GPUPhoto Credit(This article was first published ...,0.588091,0.597690,0.601689,0.599742,0.619534,0.636450,0.569693,0.589725,0.570566,0.572293,0.524144,0.570000,0.568478,0.534849,0.563137
8,HOLMES: Health OnLine Model Ensemble Serving for Deep Learning Models in Intensive Care Units,HOLMES is tested on risk prediction task on pediatric cardio ICU data with above 95% prediction ...,0.560064,0.594392,0.559752,0.530292,0.560940,0.542139,0.546098,0.573157,0.552135,0.544590,0.530991,0.522977,0.529952,0.543969,0.558750
9,[Notes] Training Question Answering Models From Synthetic Data,Generate questions and answers from both real and synthetic contextsPhoto Credit(This post was o...,0.561783,0.582028,0.574779,0.578210,0.638708,0.628183,0.615979,0.570093,0.579988,0.532942,0.568818,0.611583,0.583766,0.532249,0.530687
